In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.sklearn import log_model
import psycopg2
import json
import matplotlib.pyplot as plt
import io
from minio import Minio
import seaborn as sns
import base64
import ipaddress


In [2]:
df = pd.read_csv('./Dataset/fraudInSingapore.csv')
df

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,Latitude,Longitude,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,IsFraud
0,fb24da82-d57a-4cc0-8934-79103677a302,8d193b0d-04e1-4aef-a5da-408d628cdf4a,22509,2020-01-19,Cash-Out,West Singapore,1.441040,103.656647,7f8b2a61-d3b6-4017-843d-27ca9a01e50c,173.30.1.30,453ca213-5a95-4503-a701-ff5c30296e1d,ATM,41,Teacher,219,2,1763031,2020-01-16,1
1,c092b9de-f20d-43ba-b9d3-3ca0d8a65273,09da15a4-743d-4fce-9ed2-d15a13aeabfc,398620,2025-12-29,Cash-In,North West Singapore,1.436274,103.640656,ecc9cc86-9ccb-4457-9ee2-e2f9aef585da,122.138.245.125,b0f8ab0d-0261-4994-a667-9857878a5547,Mobile,24,Healthcare Worker,783,3,1464713,2025-12-25,0
2,16c774e3-ae99-4277-9c19-4f11592e3609,de1a4fce-0e92-4aa3-99a9-420a7086a8ec,176561,2024-09-22,Cash-In,North East Singapore,1.225685,103.702551,53edc407-bf5d-4058-885f-dad2df227c1d,91.209.140.210,e278e6d3-6e07-4c9b-a0a1-301c602c9cda,Mobile,53,Teacher,1377,4,1840735,2024-09-06,0
3,160a92a8-a541-4f91-a436-dd7f4f4b85eb,4402f7d4-cd38-441d-8987-338bd61ddfc8,171020,2025-04-23,Cash-In,West Singapore,1.107979,103.841777,0bf86876-8622-464e-af1d-97a943222111,70.231.71.101,4d7d6258-a728-47e6-8498-749371eb6e7d,ATM,67,Software Engineer,1281,2,624550,2025-04-18,1
4,93c27394-3a82-48d7-baca-da4ed59280a2,ba1af772-401e-49cb-b18f-9860e2bc2aca,801620,2020-12-13,Cash-Out,North East Singapore,1.345792,103.816744,400f9164-dc36-442f-a711-f192bab99e2d,190.159.5.96,9ce6f4d4-89f4-4a0b-9dd6-bb1aed9adbdf,ATM,70,Retail Worker,1385,0,79116,2020-11-25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,1a58c939-e422-4757-a28b-7c9fecbabb49,9793c9c7-2e17-497f-a0c2-b4a9bcb8f92a,93721,2023-02-27,Cash-Out,West Singapore,1.286555,103.733112,9c275a07-0937-437f-8bdd-a3e1f265f83c,14.106.1.187,8df421bc-6d12-4ed7-bd01-18bbedb700c5,Branch,66,Software Engineer,368,3,1964447,2023-02-19,0
6008,89dd7a5b-4aef-4f8b-b6e0-3ef84c9cb7e8,a46cc3b4-bca2-4cc3-b575-0aef35a79453,361755,2023-10-09,Cash-In,North West Singapore,1.393434,103.555124,83aa316e-121a-45b3-9593-2502f5246a3c,158.53.238.64,6787ef57-2561-4760-9b11-0dafbc1825d7,Branch,54,Healthcare Worker,1588,5,1558560,2023-09-26,0
6009,3054c0d8-e0db-477c-94e3-2808985b99cc,23c91f5d-3f88-4704-b99a-17922612ef27,761238,2023-01-14,Cash-In,West Singapore,1.388941,103.671703,deff3591-4b0c-4c96-95e2-fffee25fa1a6,94.12.75.249,3012d3a2-b59a-4bea-a1ff-904c92dd2b3d,Branch,58,Retail Worker,1348,1,717135,2023-01-08,0
6010,86329bd2-7ba7-4202-850c-b0a96ecf257a,a784a819-dff0-425f-b701-5802f319d83d,291929,2025-01-17,Cash-In,Central Singapore,1.468329,103.500698,079c96fd-b2c1-48d6-b599-b4f1ebace8b2,140.228.218.207,c513ce42-104d-4250-ae80-3e4012824be6,Branch,25,Banker,1717,5,1289650,2025-01-03,0


In [3]:
df.IsFraud.value_counts()

IsFraud
1    3006
0    3006
Name: count, dtype: int64

In [4]:
df.CustomerOccupation.value_counts()

CustomerOccupation
Healthcare Worker    1239
Banker               1212
Retail Worker        1199
Software Engineer    1186
Teacher              1176
Name: count, dtype: int64

In [5]:
lokasi = df.Location.unique()

In [6]:
print(len(lokasi))

5


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6012 entries, 0 to 6011
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TransactionID            6012 non-null   object 
 1   AccountID                6012 non-null   object 
 2   TransactionAmount        6012 non-null   int64  
 3   TransactionDate          6012 non-null   object 
 4   TransactionType          6012 non-null   object 
 5   Location                 6012 non-null   object 
 6   Latitude                 6012 non-null   float64
 7   Longitude                6012 non-null   float64
 8   DeviceID                 6012 non-null   object 
 9   IP Address               6012 non-null   object 
 10  MerchantID               6012 non-null   object 
 11  Channel                  6012 non-null   object 
 12  CustomerAge              6012 non-null   int64  
 13  CustomerOccupation       6012 non-null   object 
 14  TransactionDuration     

In [8]:
fraud = df[df['IsFraud']== 1]

In [9]:
fraud.describe()

,TransactionAmount,Latitude,Longitude,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,IsFraud
count,3006.000000,3006.000000,3006.000000,3006.000000,3006.000000,3006.000000,3.006000e+03,3006.0
mean,511814.537591,1.300246,103.749569,47.712575,1153.537591,2.519295,9.885249e+05,1.0
std,282485.609791,0.115984,0.145624,18.683863,402.447181,1.704099,5.742502e+05,0.0
min,512.000000,1.100141,103.500281,16.000000,14.000000,0.000000,1.203000e+03,1.0
25%,260899.500000,1.199180,103.619208,32.000000,845.250000,1.000000,4.805005e+05,1.0
50%,510000.500000,1.303438,103.751453,48.000000,1179.000000,3.000000,9.841920e+05,1.0
75%,758217.500000,1.400650,103.873865,64.000000,1495.000000,4.000000,1.481844e+06,1.0
max,999609.000000,1.499854,103.999913,80.000000,1800.000000,5.000000,1.999923e+06,1.0


In [10]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['PreviousTransactionDate'] = pd.to_datetime(df['PreviousTransactionDate'])



In [11]:
df

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,Latitude,Longitude,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,IsFraud
0,fb24da82-d57a-4cc0-8934-79103677a302,8d193b0d-04e1-4aef-a5da-408d628cdf4a,22509,2020-01-19,Cash-Out,West Singapore,1.441040,103.656647,7f8b2a61-d3b6-4017-843d-27ca9a01e50c,173.30.1.30,453ca213-5a95-4503-a701-ff5c30296e1d,ATM,41,Teacher,219,2,1763031,2020-01-16,1
1,c092b9de-f20d-43ba-b9d3-3ca0d8a65273,09da15a4-743d-4fce-9ed2-d15a13aeabfc,398620,2025-12-29,Cash-In,North West Singapore,1.436274,103.640656,ecc9cc86-9ccb-4457-9ee2-e2f9aef585da,122.138.245.125,b0f8ab0d-0261-4994-a667-9857878a5547,Mobile,24,Healthcare Worker,783,3,1464713,2025-12-25,0
2,16c774e3-ae99-4277-9c19-4f11592e3609,de1a4fce-0e92-4aa3-99a9-420a7086a8ec,176561,2024-09-22,Cash-In,North East Singapore,1.225685,103.702551,53edc407-bf5d-4058-885f-dad2df227c1d,91.209.140.210,e278e6d3-6e07-4c9b-a0a1-301c602c9cda,Mobile,53,Teacher,1377,4,1840735,2024-09-06,0
3,160a92a8-a541-4f91-a436-dd7f4f4b85eb,4402f7d4-cd38-441d-8987-338bd61ddfc8,171020,2025-04-23,Cash-In,West Singapore,1.107979,103.841777,0bf86876-8622-464e-af1d-97a943222111,70.231.71.101,4d7d6258-a728-47e6-8498-749371eb6e7d,ATM,67,Software Engineer,1281,2,624550,2025-04-18,1
4,93c27394-3a82-48d7-baca-da4ed59280a2,ba1af772-401e-49cb-b18f-9860e2bc2aca,801620,2020-12-13,Cash-Out,North East Singapore,1.345792,103.816744,400f9164-dc36-442f-a711-f192bab99e2d,190.159.5.96,9ce6f4d4-89f4-4a0b-9dd6-bb1aed9adbdf,ATM,70,Retail Worker,1385,0,79116,2020-11-25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,1a58c939-e422-4757-a28b-7c9fecbabb49,9793c9c7-2e17-497f-a0c2-b4a9bcb8f92a,93721,2023-02-27,Cash-Out,West Singapore,1.286555,103.733112,9c275a07-0937-437f-8bdd-a3e1f265f83c,14.106.1.187,8df421bc-6d12-4ed7-bd01-18bbedb700c5,Branch,66,Software Engineer,368,3,1964447,2023-02-19,0
6008,89dd7a5b-4aef-4f8b-b6e0-3ef84c9cb7e8,a46cc3b4-bca2-4cc3-b575-0aef35a79453,361755,2023-10-09,Cash-In,North West Singapore,1.393434,103.555124,83aa316e-121a-45b3-9593-2502f5246a3c,158.53.238.64,6787ef57-2561-4760-9b11-0dafbc1825d7,Branch,54,Healthcare Worker,1588,5,1558560,2023-09-26,0
6009,3054c0d8-e0db-477c-94e3-2808985b99cc,23c91f5d-3f88-4704-b99a-17922612ef27,761238,2023-01-14,Cash-In,West Singapore,1.388941,103.671703,deff3591-4b0c-4c96-95e2-fffee25fa1a6,94.12.75.249,3012d3a2-b59a-4bea-a1ff-904c92dd2b3d,Branch,58,Retail Worker,1348,1,717135,2023-01-08,0
6010,86329bd2-7ba7-4202-850c-b0a96ecf257a,a784a819-dff0-425f-b701-5802f319d83d,291929,2025-01-17,Cash-In,Central Singapore,1.468329,103.500698,079c96fd-b2c1-48d6-b599-b4f1ebace8b2,140.228.218.207,c513ce42-104d-4250-ae80-3e4012824be6,Branch,25,Banker,1717,5,1289650,2025-01-03,0


In [12]:
df['TransactionDate'] = df['TransactionDate'].astype('int64') // 10**9
df['PreviousTransactionDate'] = df['PreviousTransactionDate'].astype('int64') // 10**9

In [13]:
def ip_to_float(ip):
    return float(int(ipaddress.IPv4Address(ip)))

df['IP Address'] = df['IP Address'].apply(ip_to_float)



In [14]:


label_encoder = LabelEncoder()

df['TransactionType'] = label_encoder.fit_transform(df['TransactionType'])

# Cetak mapping kategori ke angka
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Mapping kategori ke angka (type):", mapping)

Mapping kategori ke angka (type): {'Cash-In': np.int64(0), 'Cash-Out': np.int64(1)}


In [15]:


label_encoder = LabelEncoder()

df['Location'] = label_encoder.fit_transform(df['Location'])

# Cetak mapping kategori ke angka
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Mapping kategori ke angka (type):", mapping)

Mapping kategori ke angka (type): {'Central Singapore': np.int64(0), 'East Singapore': np.int64(1), 'North East Singapore': np.int64(2), 'North West Singapore': np.int64(3), 'West Singapore': np.int64(4)}


In [16]:
label_encoder = LabelEncoder()

df['Channel'] = label_encoder.fit_transform(df['Channel'])

# Cetak mapping kategori ke angka
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Mapping kategori ke angka (type):", mapping)

Mapping kategori ke angka (type): {'ATM': np.int64(0), 'Branch': np.int64(1), 'Mobile': np.int64(2)}


In [17]:
label_encoder = LabelEncoder()

df['CustomerOccupation'] = label_encoder.fit_transform(df['CustomerOccupation'])

# Cetak mapping kategori ke angka
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Mapping kategori ke angka (type):", mapping)

Mapping kategori ke angka (type): {'Banker': np.int64(0), 'Healthcare Worker': np.int64(1), 'Retail Worker': np.int64(2), 'Software Engineer': np.int64(3), 'Teacher': np.int64(4)}


In [18]:
df

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,Latitude,Longitude,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,IsFraud
0,fb24da82-d57a-4cc0-8934-79103677a302,8d193b0d-04e1-4aef-a5da-408d628cdf4a,22509,1579392000,1,4,1.441040,103.656647,7f8b2a61-d3b6-4017-843d-27ca9a01e50c,2.904425e+09,453ca213-5a95-4503-a701-ff5c30296e1d,0,41,4,219,2,1763031,1579132800,1
1,c092b9de-f20d-43ba-b9d3-3ca0d8a65273,09da15a4-743d-4fce-9ed2-d15a13aeabfc,398620,1766966400,0,3,1.436274,103.640656,ecc9cc86-9ccb-4457-9ee2-e2f9aef585da,2.055927e+09,b0f8ab0d-0261-4994-a667-9857878a5547,2,24,1,783,3,1464713,1766620800,0
2,16c774e3-ae99-4277-9c19-4f11592e3609,de1a4fce-0e92-4aa3-99a9-420a7086a8ec,176561,1726963200,0,2,1.225685,103.702551,53edc407-bf5d-4058-885f-dad2df227c1d,1.540460e+09,e278e6d3-6e07-4c9b-a0a1-301c602c9cda,2,53,4,1377,4,1840735,1725580800,0
3,160a92a8-a541-4f91-a436-dd7f4f4b85eb,4402f7d4-cd38-441d-8987-338bd61ddfc8,171020,1745366400,0,4,1.107979,103.841777,0bf86876-8622-464e-af1d-97a943222111,1.189562e+09,4d7d6258-a728-47e6-8498-749371eb6e7d,0,67,3,1281,2,624550,1744934400,1
4,93c27394-3a82-48d7-baca-da4ed59280a2,ba1af772-401e-49cb-b18f-9860e2bc2aca,801620,1607817600,1,2,1.345792,103.816744,400f9164-dc36-442f-a711-f192bab99e2d,3.198093e+09,9ce6f4d4-89f4-4a0b-9dd6-bb1aed9adbdf,0,70,2,1385,0,79116,1606262400,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,1a58c939-e422-4757-a28b-7c9fecbabb49,9793c9c7-2e17-497f-a0c2-b4a9bcb8f92a,93721,1677456000,1,4,1.286555,103.733112,9c275a07-0937-437f-8bdd-a3e1f265f83c,2.418283e+08,8df421bc-6d12-4ed7-bd01-18bbedb700c5,1,66,3,368,3,1964447,1676764800,0
6008,89dd7a5b-4aef-4f8b-b6e0-3ef84c9cb7e8,a46cc3b4-bca2-4cc3-b575-0aef35a79453,361755,1696809600,0,3,1.393434,103.555124,83aa316e-121a-45b3-9593-2502f5246a3c,2.654335e+09,6787ef57-2561-4760-9b11-0dafbc1825d7,1,54,1,1588,5,1558560,1695686400,0
6009,3054c0d8-e0db-477c-94e3-2808985b99cc,23c91f5d-3f88-4704-b99a-17922612ef27,761238,1673654400,0,4,1.388941,103.671703,deff3591-4b0c-4c96-95e2-fffee25fa1a6,1.577864e+09,3012d3a2-b59a-4bea-a1ff-904c92dd2b3d,1,58,2,1348,1,717135,1673136000,0
6010,86329bd2-7ba7-4202-850c-b0a96ecf257a,a784a819-dff0-425f-b701-5802f319d83d,291929,1737072000,0,0,1.468329,103.500698,079c96fd-b2c1-48d6-b599-b4f1ebace8b2,2.363808e+09,c513ce42-104d-4250-ae80-3e4012824be6,1,25,0,1717,5,1289650,1735862400,0


In [24]:
print(df.columns)

Index(['TransactionID', 'AccountID', 'TransactionAmount', 'TransactionDate',
       'TransactionType', 'Location', 'Latitude', 'Longitude', 'DeviceID',
       'IP Address', 'MerchantID', 'Channel', 'CustomerAge',
       'CustomerOccupation', 'TransactionDuration', 'LoginAttempts',
       'AccountBalance', 'PreviousTransactionDate', 'IsFraud'],
      dtype='object')


In [19]:
X = df.drop(columns=['TransactionID', 'AccountID','DeviceID', 'MerchantID', 'IsFraud'])  # Fitur
y = df['IsFraud']

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6012 entries, 0 to 6011
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TransactionAmount        6012 non-null   int64  
 1   TransactionDate          6012 non-null   int64  
 2   TransactionType          6012 non-null   int64  
 3   Location                 6012 non-null   int64  
 4   Latitude                 6012 non-null   float64
 5   Longitude                6012 non-null   float64
 6   IP Address               6012 non-null   float64
 7   Channel                  6012 non-null   int64  
 8   CustomerAge              6012 non-null   int64  
 9   CustomerOccupation       6012 non-null   int64  
 10  TransactionDuration      6012 non-null   int64  
 11  LoginAttempts            6012 non-null   int64  
 12  AccountBalance           6012 non-null   int64  
 13  PreviousTransactionDate  6012 non-null   int64  
dtypes: float64(3), int64(11)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:

# 3. Membuat dan melatih model RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 4. Evaluasi model
y_pred = model.predict(X_test)

In [23]:
import mlflow
from mlflow.tracking import MlflowClient
import io
import base64
import psycopg2
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from minio import Minio

# Konfigurasi MinIO client
minio_client = Minio(
    "localhost:9000",  # Ganti dengan endpoint MinIO yang sesuai
    access_key="minioadmin",  # Ganti dengan access key
    secret_key="minioadmin",  # Ganti dengan secret key
    secure=False
)

# Mengecek apakah bucket sudah ada, jika belum, buat bucket
bucket_name = "mlflow-bucket"
if not minio_client.bucket_exists(bucket_name):
    minio_client.make_bucket(bucket_name)
    print(f"Bucket '{bucket_name}' dibuat.")
else:
    print(f"Bucket '{bucket_name}' sudah ada.")

# Melakukan prediksi dengan model
y_pred = model.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)

# Membuat classification report
report = classification_report(y_test, y_pred)

# Menyimpan confusion matrix ke buffer
cm = confusion_matrix(y_test, y_pred)
buf = io.BytesIO()
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Non-Fraud", "Fraud"], yticklabels=["Non-Fraud", "Fraud"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.savefig(buf, format="png")  # Menyimpan gambar ke dalam buffer
buf.seek(0)  # Mengembalikan posisi buffer ke awal

# Konversi gambar ke base64
image_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
buf.close()

experiment_name = "FraudDetectionExperiment"
mlflow.set_experiment(experiment_name)

# Setup MLflow client untuk berinteraksi dengan model registry
client = MlflowClient()

# Nama model yang digunakan
model_name = "FraudDetectionModel"

# Memeriksa apakah model sudah terdaftar
try:
    # Mendapatkan informasi model yang terdaftar
    registered_model = client.get_registered_model(model_name)
    print(f"Model {model_name} sudah terdaftar.")
    model_versions = registered_model.latest_versions
    if model_versions:
        latest_version = model_versions[0].version
        print(f"Model versi terbaru: {latest_version}")
    else:
        print(f"Model {model_name} belum memiliki versi.")
except mlflow.exceptions.MlflowException:
    print(f"Model {model_name} belum terdaftar di MLflow.")
    # Anda dapat memilih untuk mendaftar model baru jika perlu

# Menggunakan MLflow untuk log eksperimen dan model
with mlflow.start_run(run_name="RandomForestModel"):
    # Menyimpan model ke MinIO
    mlflow.sklearn.log_model(model, artifact_path="model", registered_model_name=model_name)

    # Mencatat parameter dan metrik
    mlflow.log_param("model_type", "RandomForestClassifier")
    mlflow.log_metric("accuracy", accuracy)
    
    # Mencatat classification report (sebagai teks)
    mlflow.log_text(report, "classification_report.txt")
    
    # Mengonversi kembali base64 ke byte stream dan meng-upload ke MinIO
    image_data = base64.b64decode(image_base64)
    minio_client.put_object(bucket_name, "confusion_matrix.png", io.BytesIO(image_data), len(image_data))
    print("File confusion matrix berhasil di-upload ke MinIO.")

    # Menyimpan metrik eksperimen ke PostgreSQL
    conn = psycopg2.connect(
        dbname="fraud", user="postgres", password="12345678", host="localhost", port="5432"
    )
    cur = conn.cursor()

    # Membuat tabel jika belum ada
    create_table_query = """
    CREATE TABLE IF NOT EXISTS experiments (
        id SERIAL PRIMARY KEY,
        experiment_name VARCHAR(255),
        params JSONB,
        metrics JSONB,
        run_id VARCHAR(255),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    cur.execute(create_table_query)
    conn.commit()

    # Menyimpan data eksperimen
    experiment_data = {
        "experiment_name": "RandomForestModel",
        "params": {"model_type": "RandomForestClassifier"},
        "metrics": {"accuracy": accuracy},
        "run_id": mlflow.active_run().info.run_id
    }

    cur.execute("""
        INSERT INTO experiments (experiment_name, params, metrics, run_id)
        VALUES (%s, %s, %s, %s);
    """, (
        experiment_data["experiment_name"], 
        json.dumps(experiment_data["params"]),
        json.dumps(experiment_data["metrics"]),
        experiment_data["run_id"]
    ))

    conn.commit()
    cur.close()
    conn.close()

    print("Model sudah disimpan dan didaftarkan ke MLflow dengan akurasi:", accuracy)

    # Jika akurasi lebih dari 90%, ubah stage model menjadi "Production"
    if accuracy > 0.90:
        # Mengambil versi terbaru dari model
        latest_version = client.get_latest_versions(model_name)[0].version
        
        # Mengubah stage model ke "Production"
        client.transition_model_version_stage(
            model_name, latest_version, stage="Production"
        )
        print(f"Model versi {latest_version} berhasil dipromosikan ke stage 'Production'.")


MaxRetryError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /mlflow-bucket?location= (Caused by ProtocolError('Connection aborted.', BadStatusLine('ÿ\x00\x00\x00\x00\x00\x00\x00\x01\x7fl\x01calhost:9000\r\n')))

In [ ]:

# # Melakukan prediksi dengan model
# y_pred = model.predict(X_test)

# # Menghitung akurasi
# accuracy = accuracy_score(y_test, y_pred)

# # Membuat classification report
# report = classification_report(y_test, y_pred)


# # Menggunakan MLflow untuk log eksperimen dan model
# with mlflow.start_run(run_name="RandomForestModel"):
#     # Menyimpan model ke MinIO
#     log_model(model, artifact_path="model", registered_model_name="FraudDetectionModel")
    
#     # Mencatat parameter dan metrik
#     mlflow.log_param("model_type", "RandomForestClassifier")
    
#     # Mencatat metrik akurasi
#     mlflow.log_metric("accuracy", accuracy)
    
#     # Mencatat classification report (sebagai teks)
#     mlflow.log_text(report, "classification_report.txt")
    
   

In [ ]:
# print("Akurasi:", accuracy)
# print("Classification Report:", report)

In [ ]:
# # Menampilkan confusion matrix
# cm = confusion_matrix(y_test, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Non-Fraud', 'Fraud'])
# disp.plot(cmap='Blues')